In [1]:
from pyquery import PyQuery
import requests

In [2]:
page = "https://www.pagina12.com.ar/autores/861-juan-forn"


In [3]:
import json
viejos = json.load(open("contratapas.json"))

In [5]:
viejos = [d["titulo"] for d in viejos]

In [21]:
url = "https://www.pagina12.com.ar/autores/861-juan-forn"
page = 0
links = []
while True: 
    params = {"page": page} if page else None
    listing = requests.get(url, params=params).content
    print(page)
    pq = PyQuery(listing)
    pq.make_links_absolute('http://www.pagina12.com.ar/')
    
    new_links = [a.attrib['href'] for a in pq('h4 a, h3 a') if a.text not in viejos]
    if not new_links:
        break
        
    links.extend(new_links)
    page += 1    
    

0
1
2
3
4
5
6
7
8
9


In [22]:
len(links)

95

In [44]:
def nota_pagina(url):
    pq = PyQuery(requests.get(url).content)
    pq.make_links_absolute('http://www.pagina12.com.ar/')
    data = {}
    data['contenido'] = pq('.article-text').html()
    data['foto'] = pq('.article-main-media-image img.show-for-large-only').attr('data-src')
    data['titulo'] = pq('h1.article-title').text()
    data['fecha'] = pq('div.time').text()
    data['autor'] = pq('.article-author:first a').text()
    return data

In [45]:
data = [nota_pagina(link) for link in links]

In [46]:
from ebooklib import epub

book = epub.EpubBook()

# set metadata
book.set_identifier('forn_20210420')
book.set_title('Juan Forn - Contratapas 2')
book.set_language('es')

book.add_author('Juan Forn')

chapters = []
for i, ch in enumerate(data):
    # create chapter
    c1 = epub.EpubHtml(title=ch['titulo'], file_name='chap_%02i.xhtml' % i)
    c1.content=u'<h1>{titulo}</h1>{contenido}<p class="small"><i>Publicado el {fecha}</p>'.format(**ch)
    # add chapter
    book.add_item(c1)
    chapters.append(c1)


# add default NCX and Nav file
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# define CSS style

# define css style
style = '''
@namespace epub "http://www.idpf.org/2007/ops";
body {
font-family: Cambria, Liberation Serif, Bitstream Vera Serif, Georgia, Times, Times New Roman, serif;
}
h1 {
text-align: left;
text-transform: uppercase;
font-weight: 200;
}
ol {
list-style-type: none;
}
ol > li:first-child {
margin-top: 0.3em;
}
nav[epub|type~='toc'] > ol > li > ol {
list-style-type:square;
}
nav[epub|type~='toc'] > ol > li > ol > li {
margin-top: 0.3em;
}
p.small {font-size: 0.7em}
'''
# add css file
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)
book.add_item(nav_css)
# create spine
book.spine = ['nav'] + chapters

# create epub file
epub.write_epub('forn_contratapas_2.epub', book, {})

In [47]:
!ebook-viewer forn_contratapas.epub

InputFormatPlugin: EPUB Input running
on /home/tin/lab/forn/test.epub
load glyph failed err=6 face=0x31c6230, glyph=1912
load glyph failed err=6 face=0x3b5a8f0, glyph=1912
Segmentation fault (core dumped)


In [58]:
import json

In [59]:
json.dump(data, open('contratapas_2.json', 'w'))